In [ ]:
import fastf1
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import warnings
from pathlib import Path
import seaborn as sns

warnings.filterwarnings('ignore')

fastf1.Cache.enable_cache(os.getcwd()) # replace with ur own folder path to make a cache for faster loading

# Window size (range of the sliding window used in delta angle calculation)
N = 40 # (40 + 1 + 40  = 81 data points)

# Max and Min delta angle for a data point to be classified as being part of a corner
CORNER_THRESHOLD = 0.5

In [ ]:
year = 2025
schedule = fastf1.get_event_schedule(year)
gp_events = schedule[schedule['EventName'].str.contains("Grand Prix")]

for event in gp_events['EventName']:
    session = fastf1.get_session(year, event, 'R')
    session.load()

    for driver_number in session.drivers:
        driver_data = session.get_driver(driver_number)
        driver = driver_data['Abbreviation']

        telemetry_data = session.laps.pick_driver(driver).get_telemetry()
        telemetry_data = telemetry_data[['Time', 'X', 'Y', 'Z', 'Speed']]

        laps_data = session.laps[['Time', 'LapNumber', 'LapStartTime', 'LapTime']]
        laps_data = laps_data.sort_values('LapStartTime')

        # Create an array of lap start times and lap numbers
        lap_starts = laps_data['LapStartTime'].values
        lap_ends = laps_data['Time'].values
        lap_numbers = laps_data['LapNumber'].values

        # Use searchsorted to find the lap index for each telemetry time
        idx = np.searchsorted(lap_starts, telemetry_data['Time'], side='right') - 1

        # Make sure idx is valid
        idx[idx < 0] = 0
        idx[idx >= len(lap_numbers)] = len(lap_numbers)-1

        # Assign LapNumber only if telemetry is within lap interval
        telemetry_data['LapNumber'] = np.where(
            (telemetry_data['Time'] >= lap_starts[idx]) & (telemetry_data['Time'] < lap_ends[idx]),
            lap_numbers[idx],
            np.nan)

        telemetry_data = telemetry_data.dropna(subset=['LapNumber'])
        telemetry_data['LapNumber'] = telemetry_data['LapNumber'].astype(int)

        dx1 = telemetry_data['X'] - telemetry_data['X'].shift(N)
        dy1 = telemetry_data['Y'] - telemetry_data['Y'].shift(N)
        dx2 = telemetry_data['X'].shift(-N) - telemetry_data['X']
        dy2 = telemetry_data['Y'].shift(-N) - telemetry_data['Y']

        telemetry_data['Delta_Angle'] = np.arctan2(dx1*dy2 - dy1*dx2, dx1*dx2 + dy1*dy2)
        telemetry_data['Delta_Angle'] = telemetry_data['Delta_Angle'].fillna(0)

        telemetry_data['IsCorner'] = (telemetry_data['Delta_Angle'] > CORNER_THRESHOLD) | (telemetry_data['Delta_Angle'] < -CORNER_THRESHOLD)
        
        folder = f"{event}"
        dataFileName = f"{driver}.csv"
        filepath = os.path.join(folder, dataFileName)
        os.makedirs(folder, exist_ok=True)

        telemetry_data.to_csv(filepath)

In [ ]:
root_directory = Path(os.getcwd())

for file in root_directory.rglob('*.csv'):
    telemetry_data = pd.read_csv(file)
    plt.scatter(telemetry_data['X'],
                telemetry_data['Y'],
                c=telemetry_data['Delta_Angle'],
                cmap='viridis')
    cb = plt.colorbar()

    figFileName = f"{driver}_deltaAngle.png"
    plt.savefig(f"{file.parent}/{figFileName}")

    sns.scatterplot(x=telemetry_data['X'],
                y=telemetry_data['Y'],
                hue=telemetry_data['IsCorner'],
                palette={False: "blue", True: "red"},
                edgecolor='none')
    cb.remove()

    figFileName = f"{driver}_isCorner.png"
    plt.savefig(f"{file.parent}/{figFileName}")

In [ ]:
root_directory = Path(os.getcwd())

for file in root_directory.rglob('*.csv'):
    telemetry_data = pd.read_csv(file)
    lap_stats = (
        telemetry_data
        .groupby(["LapNumber", "IsCorner"])
        .agg(
            AvgSpeed=("Speed", "mean"),
            MaxSpeed=("Speed", "max"),
            MinSpeed=("Speed", "min"),
            Count=("Speed", "size")
        )
        .reset_index()
    )

    lap_stats = lap_stats.pivot(
        index="LapNumber", 
        columns="IsCorner", 
        values=["AvgSpeed", "MaxSpeed", "MinSpeed", "Count"]
    )

    lap_stats.columns = [f"{stat}_{'Corner' if corner else 'Straight'}" 
                        for stat, corner in lap_stats.columns]

    lap_stats = lap_stats.reset_index()

    lap_stats.to_csv(f"{file.parent}/{file.stem}_lapStats.csv")